In [1]:
%load_ext nb_black

ModuleNotFoundError: No module named 'nb_black'

In [2]:
import json
import httpx
import string
import secrets

from time import sleep
from pydantic import BaseModel

from getpass import getpass

In [3]:
class DeploymentContext(BaseModel):
    env: dict = {}

In [4]:
base_url = "http://localhost:8001"
username = "jochen"
password = getpass()
service = "single_cast_hosting"

# get access_token
client = httpx.Client(base_url=base_url)
r = client.post("/token", data={"username": username, "password": password})
access_token = r.json()["access_token"]
access_token_headers = {"authorization": f"Bearer {access_token}"}

# get service token
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers={"authorization": f"Bearer {service_token}"}
client.headers = headers

 ········


# Deploy

In [4]:
def show_details(details_url):
    r = client.get(details_url, headers=headers)
    rdata = r.json()
    len_steps = len([s for s in rdata["steps"] if s["state"] == "success"])
    started = rdata["started"]
    finished = rdata["finished"]
    print(f"{len_steps} {started} {finished}")

## Create Service

In [12]:
alphabet = string.ascii_letters + string.digits
database_password = ''.join(secrets.choice(alphabet) for i in range(20))
secret_key = "".join(secrets.choice(alphabet) for i in range(32))
fqdn = "happypodcast.staging.django-cast.com"
underscored_fqdn = fqdn.replace(".", "_")
site_id = f"cast_{underscored_fqdn}"
env = {
    "fqdn": fqdn,
    "site_id": site_id,
    "user_name": "cast_1",
    "database_name": site_id,
    "database_user": site_id,
    "database_password": database_password,
    "secret_key": secret_key,
    "port": 10001,
    "settings_file_name": site_id
}
env

{'fqdn': 'happypodcast.staging.django-cast.com',
 'site_id': 'cast_happypodcast_staging_django-cast_com',
 'user_name': 'cast_1',
 'database_name': 'cast_happypodcast_staging_django-cast_com',
 'database_user': 'cast_happypodcast_staging_django-cast_com',
 'database_password': 'ghdZcfFv9IVeVoOXPG63',
 'secret_key': 'FfmsKCbmhZAz8to3vjafg0kdj1huEJdr',
 'port': 10001,
 'settings_file_name': 'cast_happypodcast_staging_django-cast_com'}

### Get Service Token

In [20]:
data = {"service": service, "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers = {"authorization": f"Bearer {service_token}"}

<IPython.core.display.Javascript object>

In [21]:
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

# for i in range(3):
#     show_details(details_url)
#     sleep(1)

{'id': 160, 'service_id': 6, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-01-28T06:06:36.056875', 'finished': None, 'context': {'env': {'fqdn': 'jochen.staging.django-cast.com', 'site_id': 'cast_jochen_staging_django-cast_com', 'user_name': 'cast_1', 'database_name': 'cast_jochen_staging_django-cast_com', 'database_user': 'cast_jochen_staging_django-cast_com', 'database_password': 'thQYxfqIczyj67TaUKQC', 'secret_key': 'fP1ZOi14a9rsQ5kRASnS0ZXhFycpnzMf', 'port': 10001, 'settings_file_name': 'cast_jochen_staging_django-cast_com'}}, 'type': 'deployment', 'deleted': False, 'details': 'http://localhost:8000/deployments/160'}


<IPython.core.display.Javascript object>

## Remove Service

### Get Service Token

In [13]:
data = {"service": "remove_single_cast_hosting", "origin": "notebook"}
r = client.post("/service-token", json=data, headers=access_token_headers)
service_token = r.json()["service_token"]
headers={"authorization": f"Bearer {service_token}"}

In [14]:
context = DeploymentContext(env=env)
r = client.post("/deployments/", json=context.dict(), headers=headers)
rdata = r.json()
print(rdata)
details_url = r.json()["details"]

{'id': 192, 'service_id': 7, 'origin': 'notebook', 'user': 'jochen', 'started': '2022-01-30T16:03:02.297392', 'finished': None, 'context': {'env': {'fqdn': 'happypodcast.staging.django-cast.com', 'site_id': 'cast_happypodcast_staging_django-cast_com', 'user_name': 'cast_1', 'database_name': 'cast_happypodcast_staging_django-cast_com', 'database_user': 'cast_happypodcast_staging_django-cast_com', 'database_password': 'ghdZcfFv9IVeVoOXPG63', 'secret_key': 'FfmsKCbmhZAz8to3vjafg0kdj1huEJdr', 'port': 10001, 'settings_file_name': 'cast_happypodcast_staging_django-cast_com'}}, 'type': 'deployment', 'deleted': False, 'details': 'http://localhost:8001/deployments/192'}


In [35]:
context.env

{'fqdn': 'foobar.staging.wersdoerfer.de',
 'database_name': 'cast_foobar.staging.wersdoerfer.de',
 'database_user': 'cast_foobar.staging.wersdoerfer.de',
 'database_password': '6It0n1QGoMM4GtAkGmRr',
 'secret_key': '2oCTxJIo9YBpWquiVG5Ks3cJHcIPKZUB',
 'port': 10001,
 'settings_file_name': 'foobar_staging_wersdoerfer_de'}

<IPython.core.display.Javascript object>

# Stuff

In [31]:
foo = {"bar": "baz"}

<IPython.core.display.Javascript object>

In [32]:
dict(foo, blub="bla")

{'bar': 'baz', 'blub': 'bla'}

<IPython.core.display.Javascript object>

In [33]:
dict(foo, bar="blub")

{'bar': 'blub'}

<IPython.core.display.Javascript object>